In [26]:
import pandas as pd
import glob

## Extract BNF from Original

In [27]:
df = pd.read_csv('/Users/carboni/Downloads/Gallica fashion.csv')

In [3]:
#sru_nca = pd.read_csv('SRU_BNF_nca.csv')

In [ ]:
filtered_df = df[df['URL'].str.contains("https://gallica.bnf.fr/")].copy()

In [31]:
filtered_df.head()

,List,OK VC,VC_Title,VC_Journal Type,VC_City,VC_Country,ark
0,Gallica,OK ?,"Art, goût, beauté : feuillets de l'élégance fé...",Fashion,NaN,NaN,ark:/12148/cb327024113/date
1,Perfume Drinker list,Trop vieux,"Cabinet des modes, ou les Modes nouvelles, déc...",Fashion,Paris,France,ark:/12148/cb327349107/date
2,Gallica,OK ?,Chapeaux,Fashion,NaN,NaN,ark:/12148/cb45736932j/date
3,Perfume Drinker list,OK ?,Chapeaux d'Été,Fashion,Paris,France,ark:/12148/cb32740151v/date
4,Gallica,OK ?,Chapeaux d'hiver,Fashion,NaN,NaN,ark:/12148/cb327401526/date


In [5]:
filtered_no_SRU = filtered_df[~filtered_df['URL'].str.contains("https://gallica.bnf.fr/services/engine/search/sru?")].copy()

In [6]:
combined_df = pd.concat([filtered_no_SRU, sru_nca], ignore_index=True)

In [7]:
combined_df.head()

,Titre,URL,source
0,L'Art décoratif,https://gallica.bnf.fr/iiif/12148/bpt6k1204557...,barbara_iif.csv
1,Action francaise,https://gallica.bnf.fr/ark:/12148/cb326819451/...,celine_1.csv
2,L'Aurore,https://gallica.bnf.fr/ark:/12148/cb32706846t/...,celine_1.csv
3,L'Auto,https://gallica.bnf.fr/ark:/12148/cb327071375/...,celine_1.csv
4,Le Soir,https://gallica.bnf.fr/ark:/12148/cb32738400h/...,celine_1.csv


In [8]:
combined_df["URL"] = combined_df["URL"].str.replace('https://gallica.bnf.fr/iiif/12148/bpt6k1204557d/manifest.json', 'https://gallica.bnf.fr/ark:/12148/bpt6k1204557d/date', regex=False)

In [34]:
mask = combined_df['URL'].str.contains(r'date&|date\.item')
combined_df.loc[mask, 'URL'] = combined_df.loc[mask, 'URL'].str.replace(r'date&.*$|date\.item.*$', 'date', regex=True)

In [35]:
combined_df["ark"] = combined_df["URL"].str.replace('https://gallica.bnf.fr/ark:/12148/', '', regex=False).str.replace('/date', '', regex=False)

In [36]:
combined_df.to_csv('filtered_marie_bnf.csv', index=False)

In [44]:
combined_df.head()

,List,OK VC,VC_Title,VC_Journal Type,VC_City,VC_Country,ark,URL
0,Gallica,OK ?,"Art, goût, beauté : feuillets de l'élégance fé...",Fashion,NaN,NaN,cb327024113,ark:/12148/cb327024113/date
1,Perfume Drinker list,Trop vieux,"Cabinet des modes, ou les Modes nouvelles, déc...",Fashion,Paris,France,cb327349107,ark:/12148/cb327349107/date
2,Gallica,OK ?,Chapeaux,Fashion,NaN,NaN,cb45736932j,ark:/12148/cb45736932j/date
3,Perfume Drinker list,OK ?,Chapeaux d'Été,Fashion,Paris,France,cb32740151v,ark:/12148/cb32740151v/date
4,Gallica,OK ?,Chapeaux d'hiver,Fashion,NaN,NaN,cb327401526,ark:/12148/cb327401526/date


## Match BNF arks to Corpus

In [38]:
folder_path = '/Users/carboni/Downloads/bnf'

In [39]:
keep_col = ['Notice']

In [40]:
all_files = glob.glob(folder_path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, delimiter=',', low_memory=False)
    df = df[keep_col]
    li.append(df)

df_merged = pd.concat(li, axis=0, ignore_index=True)

In [41]:
df_merged["ark"] = df_merged["Notice"].str.replace('https://gallica.bnf.fr/ark:/12148/', '', regex=False).str.replace('/date', '', regex=False)

In [42]:
df_merged.head()

,Notice,ark
0,https://gallica.bnf.fr/ark:/12148/cb32788632q/...,cb32788632q
1,https://gallica.bnf.fr/ark:/12148/cb32788632q/...,cb32788632q
2,https://gallica.bnf.fr/ark:/12148/cb32788632q/...,cb32788632q
3,https://gallica.bnf.fr/ark:/12148/cb32788632q/...,cb32788632q
4,https://gallica.bnf.fr/ark:/12148/cb32788632q/...,cb32788632q


In [45]:
combined_df["present"] = combined_df["ark"].isin(df_merged["ark"])

In [46]:
combined_df.head()

,List,OK VC,VC_Title,VC_Journal Type,VC_City,VC_Country,ark,URL,present
0,Gallica,OK ?,"Art, goût, beauté : feuillets de l'élégance fé...",Fashion,NaN,NaN,cb327024113,ark:/12148/cb327024113/date,True
1,Perfume Drinker list,Trop vieux,"Cabinet des modes, ou les Modes nouvelles, déc...",Fashion,Paris,France,cb327349107,ark:/12148/cb327349107/date,False
2,Gallica,OK ?,Chapeaux,Fashion,NaN,NaN,cb45736932j,ark:/12148/cb45736932j/date,True
3,Perfume Drinker list,OK ?,Chapeaux d'Été,Fashion,Paris,France,cb32740151v,ark:/12148/cb32740151v/date,False
4,Gallica,OK ?,Chapeaux d'hiver,Fashion,NaN,NaN,cb327401526,ark:/12148/cb327401526/date,True


In [47]:
only_new = combined_df[combined_df["present"] == False].copy()

In [48]:
only_new.head()

,List,OK VC,VC_Title,VC_Journal Type,VC_City,VC_Country,ark,URL,present
1,Perfume Drinker list,Trop vieux,"Cabinet des modes, ou les Modes nouvelles, déc...",Fashion,Paris,France,cb327349107,ark:/12148/cb327349107/date,False
3,Perfume Drinker list,OK ?,Chapeaux d'Été,Fashion,Paris,France,cb32740151v,ark:/12148/cb32740151v/date,False
9,Perfume Drinker list,Trop vieux,Gallery of Fashion,Fashion,London,United Kingdom,cb327795523,ark:/12148/cb327795523/date,False
11,Perfume Drinker list,Trop vieux,"Journal de la mode et du goût, ou Amusemens du...",Fashion,Paris,France,cb32798156j,ark:/12148/cb32798156j/date,False
25,Perfume Drinker list,Trop vieux / livre,Le Bon Genre,Fashion,Paris,France,bpt6k1054664s/f35.image,ark:/12148/bpt6k1054664s/f35.image,False


In [49]:
only_new.to_csv('bnf_to_add_marie.csv', index=False)